#### Setting notebook parameters

In [1]:
import pandas as pd

from IPython.core.display import display, HTML 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

display(HTML("<style>.container { width:90% !important; }</style>"))

#### Download the SIRENE database from data.gouv

In [ ]:
# Specify the path where to store data
PATH = "C:/Users/defra/Desktop/git/DATA/"
# PATH = ""

In [2]:
import requests
import os
import zipfile

# Url of the sirene database on data.gouv
DATA_URL = "https://www.data.gouv.fr/fr/datasets/r/4a8befd7-1d05-41cd-9e1e-b0f19c73d73b" 

# Accessing the URL
r = requests.get(DATA_URL, allow_redirects=True)

# Storing data
with open(PATH+'sirene.zip', 'wb') as f:
    f.write(r.content)

# Unzipping data
with zipfile.ZipFile(PATH+'sirene.zip', 'r') as zip_ref:
    zip_ref.extractall(PATH)

#### Data sampling

In [3]:
# Reading extracted CSV (Warning: lots of RAM might be needed)
sirene = pd.read_csv(PATH+'StockEtablissement_utf8.csv', low_memory=False)

In [4]:
# Filtering
# active businesses
sirene = sirene.query("etatAdministratifEtablissement=='A'") 
# named businesses
sirene = sirene[~sirene.denominationUsuelleEtablissement.isna()] 
# retail businesses
sirene = sirene[sirene.activitePrincipaleEtablissement.apply(lambda u:u.startswith('47'))]
# checking for an adress
sirene = sirene[~sirene.libelleCommuneEtablissement.isna()]

# Businesses' adress
sirene["adress"] = (
    sirene.numeroVoieEtablissement.fillna('') + ' ' +
    sirene.typeVoieEtablissement.fillna('') + ' ' +
    sirene.libelleVoieEtablissement.fillna('') + ' ' +
    sirene.codePostalEtablissement.fillna(0).astype(int).astype(str).replace('0','') + ' ' + 
    sirene.libelleCommuneEtablissement.fillna('')
).apply(lambda u: ' '.join(u.split()))

In [195]:
# Drawing a sample 
test = sirene.sample(n=5, random_state=0)
test

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,typeVoieEtablissement,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,adress
23728492,791068182,17,79106818200017,O,2013-02-10,01,2018.0,NaN,2020-08-25T11:01:29,True,1,NaN,7,NaN,RUE,DU QUATRE SEPTEMBRE,34500.0,BEZIERS,NaN,NaN,34032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-02-10,A,MADAME M,NaN,NaN,MADAME M,47.72A,NAFRev2,O,7 RUE DU QUATRE SEPTEMBRE 34500 BEZIERS
28544991,844021519,13,84402151900013,O,2018-12-01,NaN,NaN,NaN,2019-11-14T14:01:05,True,1,NaN,6,NaN,RUE,ROGER VAILLANT,91700.0,SAINTE-GENEVIEVE-DES-BOIS,NaN,NaN,91549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-01,A,NaN,NaN,NaN,BOOTIK ESSONNE.FR,47.82Z,NAFRev2,N,6 RUE ROGER VAILLANT 91700 SAINTE-GENEVIEVE-DE...
2598060,323473884,43,32347388400043,O,2013-06-20,NaN,NaN,NaN,2019-11-14T14:00:27,True,5,LA GEBELINIERE,135,NaN,CHE,DU COTEAU,26750.0,SAINT-PAUL-LES-ROMANS,NaN,NaN,26323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-20,A,NaN,NaN,NaN,THIERY OUTILLAGE,47.89Z,NAFRev2,N,135 CHE DU COTEAU 26750 SAINT-PAUL-LES-ROMANS
28882261,849710090,10,84971009000010,O,2019-04-13,NaN,NaN,NaN,2019-10-17T03:55:24,True,1,NaN,125,NaN,RUE,DE MEAUX,75019.0,PARIS 19,NaN,NaN,75119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-13,A,NaN,NaN,NaN,SUBSTANCES,47.25Z,NAFRev2,N,125 RUE DE MEAUX 75019 PARIS 19
16905360,493886832,69,49388683200069,O,2016-03-01,11,2018.0,NaN,2020-08-25T10:39:53,True,1,NaN,67,NaN,RUE,CHAMP DE L ORME,69100.0,VILLEURBANNE,NaN,NaN,69266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-01,A,NaN,NaN,NaN,FEN PRO - HM RENOV 26,47.52B,NAFRev2,O,67 RUE CHAMP DE L ORME 69100 VILLEURBANNE


#### Google Search Results

In [191]:
# Building search requests
test['request'] = (
    # name of the business: making sure it matches entirely
    test.denominationUsuelleEtablissement.apply(lambda u:'"'+u+'"') + '+' +
    # city of the business: making sure to search in the right city
    test.libelleCommuneEtablissement.apply(lambda u: '+'.join(u.split()))
)

In [187]:
import selenium.webdriver as webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

def get_result(request, headless=True):
    """ Google (Maps) Search of the request."""
    
    # Open web browser
    options = Options() 
    options.headless = headless # Setting options to not actually open it
    browser = webdriver.Firefox(options=options)
    
    # Accessing the search results
    GOOGLE_SEARCH_URL = "https://google.com/search?q="
    browser.get(GOOGLE_SEARCH_URL+request)

    # Agreeing to cookies
    WebDriverWait(browser,10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[src^='https://consent.google.com']")))
    WebDriverWait(browser,10).until(EC.element_to_be_clickable((By.XPATH,"//div[@id='introAgreeButton']"))).click()
    browser.switch_to.default_content() # switching back to default frame

    # Accessing Google Maps
    WebDriverWait(browser,10).until(EC.element_to_be_clickable((By.XPATH,"//*[contains(text(), 'Maps')]"))).click()

    # Getting adress, phone number and website
    browser.implicitly_wait(5) # making sure the left panel is loaded
    infos = browser.find_elements_by_class_name("ugiz4pqJLAG__button")
    # extracting data into a dict
    dict_infos = dict()
    for info in infos:
        label = info.get_attribute("aria-label")
        if (label is not None):
            label = label.split(': ')
            dict_infos[label[0]] = label[1]
    
    # Getting ratings and number of voters
    browser.implicitly_wait(5) # making sure the left panel is loaded
    try:
        # Rating
        rating = browser.find_element_by_class_name("section-star-array")
        dict_infos['Note'] = rating.get_attribute('aria-label')
        
        # Number of voters
        voters = browser.find_element_by_xpath("//button[@jsaction='pane.rating.moreReviews']")
        dict_infos['Nb votes'] = voters.get_attribute('aria-label')
    except:
        pass
    
    
    browser.close()  
    return dict_infos


In [192]:
test.request.apply(get_result).values

array([{'Adresse': '7 Rue du 4 Septembre, 34500 Béziers ', 'Numéro de téléphone': '04 67 49 02 43 ', 'Plus\xa0code': '86V8+75 Béziers', 'Note': ' 5,0\xa0étoiles ', 'Nb votes': '2\xa0avis'},
       {}, {},
       {'Adresse': '125 Rue de Meaux, 75019 Paris ', 'Plus\xa0code': 'V9MH+WP Paris'},
       {'Adresse': "67 Rue Champ de l'Orme, 69100 Villeurbanne ", 'Site Web': 'hmrenov26.com ', 'Numéro de téléphone': '04 78 80 54 46 ', 'Plus\xa0code': 'QVCW+X7 Villeurbanne', 'Note': ' 4,0\xa0étoiles ', 'Nb votes': '348\xa0avis'}],
      dtype=object)

In [2]:
# !jupyter nbconvert --to html web_score.ipynb

[NbConvertApp] Converting notebook web_score.ipynb to html
[NbConvertApp] Writing 304857 bytes to web_score.html
